### Imports

In [73]:
from bs4 import BeautifulSoup as bs
import requests
import unicodedata  # for except /xaO in title of files
import re  # for except /r/n in title of files
from urllib import request  # for download the data
import urllib.parse  # for decode utf-8 to url-encode for download from links with cyrillic
from datetime import datetime  # for cleaned date format in title of docs
# for convert doc in docx and delete doc
import os.path
import win32com.client

### Links
Здесь собираем списки с ссылками на:
* страницы РосГидромета с отчётами по годам `(links)`;  
* doc(x)-файлы стандартным BeautifulSoup `(DocLinks)`;

In [38]:
# Create a list of links with documents by year
links = list()
link = 'http://www.meteorf.ru/product/infomaterials/99/?year='
for i in range(2008, 2022):
    links.append(link+str(i))

In [39]:
# Check the list
links[:5]

['http://www.meteorf.ru/product/infomaterials/99/?year=2008',
 'http://www.meteorf.ru/product/infomaterials/99/?year=2009',
 'http://www.meteorf.ru/product/infomaterials/99/?year=2010',
 'http://www.meteorf.ru/product/infomaterials/99/?year=2011',
 'http://www.meteorf.ru/product/infomaterials/99/?year=2012']

In [40]:
# Create a list of links on all documents
DocLinks = list()
for year in links:
    page = requests.get(year)
    soup = bs(page.text)
    for ultag in soup.find_all('ul', {'class': 'links'}):
        for litag in ultag.find_all('li'):
            DocLinks.append('http://www.meteorf.ru' +
                            str(litag.find_all('a')[0]['href']))

In [41]:
# Check the list
DocLinks[-5:]

['http://www.meteorf.ru/upload/iblock/749/2021-май-месячный обзор по Российской Федерации.docx',
 'http://www.meteorf.ru/upload/iblock/089/2021-апрель-месячный обзор по Российской Федерации.docx',
 'http://www.meteorf.ru/upload/iblock/c0a/2021-март-месячный обзор по Российской Федерации.docx',
 'http://www.meteorf.ru/upload/iblock/6a5/2021-февраль-месячный обзор по Российской Федерации.docx',
 'http://www.meteorf.ru/upload/iblock/580/2021_january_rf.docx']

### Titles of docs
Собираем список с названиями документов

Для начала c помощью Links соберём сырой список заголовков. Берём мы их из **div'ов с классом lcol**

После убираем мусор в виде `\xa0` и `\r\n`, а также попавшие сюда названия <br>
html-версий отчётов (just `HTML`).

In [42]:
# Create a list of titles of documents
DocTitle = list()
for year in links:
    page = requests.get(year)
    soup = bs(page.text)
    for divtag in soup.find_all('div', {'class': 'lcol'}):
        # Clean the garbage from titles
        for spantag in divtag.find_all('span'):
            UnicodeStr = spantag.text
            ClnStr = unicodedata.normalize("NFKD", UnicodeStr)
            ClnStr = re.sub(r'\r\n', '', ClnStr)
            # Remove titles on links on "HTML version" (ha-ha) of docs
            if ClnStr != 'HTML':
                DocTitle.append(ClnStr)

Теперь готовим унификацию названий.

Опытным путём выяснили, что в стандартные названия отчётов убрали пробелы в некоторых местах или, наоборот, добавили больше одного. Чтобы не мучиться с перебором вариантов, просто replace'им пробелы, а затем заменяем получившийся набор сиволов `(remove)` на «ЗОС, » [Загрязнение Окружающей Среды]

А ещё убираем «года» с конца названия

In [43]:
# Shorten the file name
# Shorten similar text
DocTitle = [re.sub(r'\s', '', i) for i in DocTitle]  # remove all space symbols
remove = 'Обаварийном,экстремальновысокомивысокомзагрязненииокружающейсреды,\
атакжерадиационнойобстановкенатерриторииРоссиив'
for Title in DocTitle:
    if re.match(remove, Title):
        DocTitle = [i.replace((remove), 'ЗОС, ') for i in DocTitle]

# remove all 'года' from end of strings
DocTitle = [re.sub(r'года', '', i) for i in DocTitle]

In [44]:
# Check the list
DocTitle

['МесячныйобзорпотерриторииРФ(апрель)',
 'ЗОС, сентябре2008',
 'ЗОС, ноябре2008',
 'ЗОС, декабре2008',
 'ЗОС, январе2009',
 'ЗОС, феврале2009',
 'ЗОС, марте2009',
 'ЗОС, ноябре2009',
 'ЗОС, декабре2009',
 'ЗОС, январе2010',
 'Обаварийном,экстремальновысокомивысокомзагрязненииокружающейсреды,такжерадиационнойобстановкенатерриторииРоссиивфеврале2010',
 'ЗОС, марте2010',
 'ЗОС, апреле2010',
 'ЗОС, июне2010',
 'ЗОС, июле2010',
 'ЗОС, августе2010',
 'ЗОС, сентябре2010',
 'ЗОС, октябре2010',
 'ЗОС, ноябре2010',
 'ЗОС, декабре2010',
 'ЗОС, январе2011',
 'ЗОС, сентябре2011',
 'ЗОС, марте2012',
 'ЗОС, апреле2012',
 'ЗОС, июне2012',
 'ЗОС, августе2012',
 'ЗОС, октябре2012',
 'ЗОС, ноябре2012',
 'ЗОС, декабре2012',
 'ЗОС, декабре2012',
 'ЗОС, декабре2013',
 'ЗОС, ноябре2013',
 'ЗОС, октябре2013',
 'ЗОС, сентябре2013',
 'ЗОС, августе2013',
 'ЗОС, июле2013',
 'Обаварийном,экстремальновысокомивысокомзагрязненииокружающейсредынатерриторииРоссийскойФедерациис19по26июля2013',
 'Обаварийном,экс

Чуть изменив пример со Stack Overflow, заменяем все месяцы, написанные текстом, на числовое выражение. 

В итоге получаем красивое название файлов, например, `«ЗОС, 9.2013»`

In [45]:
# For replace str month to int month
RU_MONTH_VALUES = {
    'январе': '1.',
    'феврале': '2.',
    'марте': '3.',
    'апреле': '4.',
    'мае': '5.',
    'июне': '6.',
    'июле': '7.',
    'августе': '8.',
    'сентябре': '9.',
    'октябре': '10.',
    'ноябре': '11.',
    'декабре': '12.',
}


def int_value_from_ru_month(date_str):
    for k, v in RU_MONTH_VALUES.items():
        date_str = date_str.replace(k, str(v))

    return date_str


DocTitle = [int_value_from_ru_month(d) for d in DocTitle]

Проверив список, заменяем вручную те места, где текст названий всё же отличался. Было бы больше исключений — зациклил бы, но в нашем случае так быстрей.

In [46]:
# Chech the list
DocTitle

['МесячныйобзорпотерриторииРФ(апрель)',
 'ЗОС, 9.2008',
 'ЗОС, 11.2008',
 'ЗОС, 12.2008',
 'ЗОС, 1.2009',
 'ЗОС, 2.2009',
 'ЗОС, 3.2009',
 'ЗОС, 11.2009',
 'ЗОС, 12.2009',
 'ЗОС, 1.2010',
 'Обаварийном,экстремальновысокомивысокомзагрязненииокружающейсреды,такжерадиационнойобстановкенатерриторииРоссиив2.2010',
 'ЗОС, 3.2010',
 'ЗОС, 4.2010',
 'ЗОС, 6.2010',
 'ЗОС, 7.2010',
 'ЗОС, 8.2010',
 'ЗОС, 9.2010',
 'ЗОС, 10.2010',
 'ЗОС, 11.2010',
 'ЗОС, 12.2010',
 'ЗОС, 1.2011',
 'ЗОС, 9.2011',
 'ЗОС, 3.2012',
 'ЗОС, 4.2012',
 'ЗОС, 6.2012',
 'ЗОС, 8.2012',
 'ЗОС, 10.2012',
 'ЗОС, 11.2012',
 'ЗОС, 12.2012',
 'ЗОС, 12.2012',
 'ЗОС, 12.2013',
 'ЗОС, 11.2013',
 'ЗОС, 10.2013',
 'ЗОС, 9.2013',
 'ЗОС, 8.2013',
 'ЗОС, 7.2013',
 'Обаварийном,экстремальновысокомивысокомзагрязненииокружающейсредынатерриторииРоссийскойФедерациис19по26июля2013',
 'Обаварийном,экстремальновысокомивысокомзагрязненииокружающейсредынатерриторииРоссийскойФедерациивпериодс12июляпо19июля2013',
 'ЗОС, 6.2013',
 'ЗОС, 5

In [47]:
# manual cleaning of garbage not cleaned by the program
# a particular case that we did not foresee
DocTitle[10] = 'ЗОС, 2.2010'
DocTitle[0] = 'ЗОС, 4.2008'

Теперь по проверке на начало строки с названием отлавливаем те названия, отчёты в которых — не по месяцу, а по периоду (напр, с 19 по 26 июля 2013 года). Помечаем текстом `«ПЕРИОД»` в начале, чтобы не работать с файлами сразу — несмотря на наличие таких файлов, отчёты по полному месяцу есть. Но на всякий случай — скачаем. 

In [48]:
# Check the not cleaned titles for manual cleaning
for Title in DocTitle:
    if not re.match('ЗОС, ', Title):
        print(Title)
        print(DocTitle.index(Title))

Обаварийном,экстремальновысокомивысокомзагрязненииокружающейсредынатерриторииРоссийскойФедерациис19по26июля2013
36
Обаварийном,экстремальновысокомивысокомзагрязненииокружающейсредынатерриторииРоссийскойФедерациивпериодс12июляпо19июля2013
37
Обаварийном,экстремальновысокомивысокомзагрязненииокружающейсредынатерриторииРоссийскойФедерациивпериодс16по22февраля2018
101
Обаварийном,экстремальновысокомивысокомзагрязненииокружающейсредыивыявленныхслучаяхизменениярадиационнойобстановкинатерриторииРоссийскойФедерациивпериодс18по25сентября2020
120


In [49]:
DocTitle[36] = 'ПЕРИОД ЗОС, 19-26.07.2013'
DocTitle[37] = 'ПЕРИОД ЗОС, 12-19.07.2013'
DocTitle[101] = 'ПЕРИОД ЗОС, 16-22.02.2018'
DocTitle[120] = 'ПЕРИОД ЗОС, 18-25.09.2020'

In [50]:
# Chech the list
DocTitle

['ЗОС, 4.2008',
 'ЗОС, 9.2008',
 'ЗОС, 11.2008',
 'ЗОС, 12.2008',
 'ЗОС, 1.2009',
 'ЗОС, 2.2009',
 'ЗОС, 3.2009',
 'ЗОС, 11.2009',
 'ЗОС, 12.2009',
 'ЗОС, 1.2010',
 'ЗОС, 2.2010',
 'ЗОС, 3.2010',
 'ЗОС, 4.2010',
 'ЗОС, 6.2010',
 'ЗОС, 7.2010',
 'ЗОС, 8.2010',
 'ЗОС, 9.2010',
 'ЗОС, 10.2010',
 'ЗОС, 11.2010',
 'ЗОС, 12.2010',
 'ЗОС, 1.2011',
 'ЗОС, 9.2011',
 'ЗОС, 3.2012',
 'ЗОС, 4.2012',
 'ЗОС, 6.2012',
 'ЗОС, 8.2012',
 'ЗОС, 10.2012',
 'ЗОС, 11.2012',
 'ЗОС, 12.2012',
 'ЗОС, 12.2012',
 'ЗОС, 12.2013',
 'ЗОС, 11.2013',
 'ЗОС, 10.2013',
 'ЗОС, 9.2013',
 'ЗОС, 8.2013',
 'ЗОС, 7.2013',
 'ПЕРИОД ЗОС, 19-26.07.2013',
 'ПЕРИОД ЗОС, 12-19.07.2013',
 'ЗОС, 6.2013',
 'ЗОС, 5.2013',
 'ЗОС, 4.2013',
 'ЗОС, 1.2013',
 'ЗОС, 2.2013',
 'ЗОС, 3.2013',
 'ЗОС, 12.2014',
 'ЗОС, 11.2014',
 'ЗОС, 10.2014',
 'ЗОС, 9.2014',
 'ЗОС, 8.2014',
 'ЗОС, 7.2014',
 'ЗОС, 6.2014',
 'ЗОС, 5.2014',
 'ЗОС, 4.2014',
 'ЗОС, 3.2014',
 'ЗОС, 2.2014',
 'ЗОС, 1.2014',
 'ЗОС, 12.2015',
 'ЗОС, 11.2015',
 'ЗОС, 10.2015',
 'ЗОС, 9

### Error check
Проверяем сборку на ошибки

Первым делом проверяем длину списков — очевидно, их длины должны быть равны друг другу.

Однако, счёт 134 на 137. Ссылок больше.

В дальнейшем, при просмотре структуры html-страницы РГМ выясним, что:
* без названий подгружаются файлы приложений к одному из отчётов — убираем ссылки, если что, скачаем вручную. Счёт 134 на 133.
* Один из месячных отчётов не имел doc(x)-файла — это единственный отчёт, который честно поместили в html, а не сделали  ~зайца в утке~  doc внутри html'а. Убираем название из списка — докачаем вручную (в репозитории лежит PDF (версия для печати)). Счёт 133 на 133

In [56]:
# Check the len of lists
len(DocTitle), len(DocLinks)

# First iteration: 134 and 137
# Second iteration: 134 and 133 — I remove links on attachments in cell below
# Third iteration: 133 and 133 — I deleted a document title that did not have a doc(x) file.
# It has an html report, which I downloaded manually.

(134, 133)

Принтуем название отчёта и ссылку для скачивания — в большинстве случаев это помогало понять, где идёт сдвиг (напр., отчёт за май 2013 шёл в паре с названием «ЗОС, 7.2013»)

In [52]:
# Cell for check shifts
for i in range(133):
    print(DocTitle[i], '\n', DocLinks[i])

ЗОС, 4.2008 
 http://www.meteorf.ru/upload/iblock/72b/mes04r.doc
ЗОС, 9.2008 
 http://www.meteorf.ru/upload/iblock/e89/av-sent.doc
ЗОС, 11.2008 
 http://www.meteorf.ru/upload/iblock/d27/av-noy.doc
ЗОС, 12.2008 
 http://www.meteorf.ru/upload/iblock/a0a/av-dec.doc
ЗОС, 1.2009 
 http://www.meteorf.ru/upload/iblock/52e/av-r-yanv.doc
ЗОС, 2.2009 
 http://www.meteorf.ru/upload/iblock/b3a/av-r-fev.doc
ЗОС, 3.2009 
 http://www.meteorf.ru/upload/iblock/d5e/av-r-mart.doc
ЗОС, 11.2009 
 http://www.meteorf.ru/upload/iblock/d5f/av-r-noy.doc
ЗОС, 12.2009 
 http://www.meteorf.ru/upload/iblock/e14/av-r-dec.doc
ЗОС, 1.2010 
 http://www.meteorf.ru/upload/iblock/df6/av-r-yanv.doc
ЗОС, 2.2010 
 http://www.meteorf.ru/upload/iblock/812/av-r-fev.doc
ЗОС, 3.2010 
 http://www.meteorf.ru/upload/iblock/329/av-r-mart.doc
ЗОС, 4.2010 
 http://www.meteorf.ru/upload/iblock/1fc/av-r-apr.doc
ЗОС, 6.2010 
 http://www.meteorf.ru/upload/iblock/009/av-r-iun.doc
ЗОС, 7.2010 
 http://www.meteorf.ru/upload/iblock/f94/av-r-iu

Убираем ссылки на приложения:

In [53]:
# Another iteration of cleaned links and titles
# Remove links with attachments
DocLinks_wo_Pril = list()
for i in DocLinks:
    if 'prilozheniye' not in i:
        DocLinks_wo_Pril.append(i)
DocLinks = DocLinks_wo_Pril

Снова выводим на экран заголовок отчёта и ссылку на него для проверки:

In [59]:
# Cell for check shifts
for i in range(134):
    print(DocTitle[i], '\n', DocLinks_wo_Pril[i])

ЗОС, 4.2008 
 http://www.meteorf.ru/upload/iblock/72b/mes04r.doc
ЗОС, 9.2008 
 http://www.meteorf.ru/upload/iblock/e89/av-sent.doc
ЗОС, 11.2008 
 http://www.meteorf.ru/upload/iblock/d27/av-noy.doc
ЗОС, 12.2008 
 http://www.meteorf.ru/upload/iblock/a0a/av-dec.doc
ЗОС, 1.2009 
 http://www.meteorf.ru/upload/iblock/52e/av-r-yanv.doc
ЗОС, 2.2009 
 http://www.meteorf.ru/upload/iblock/b3a/av-r-fev.doc
ЗОС, 3.2009 
 http://www.meteorf.ru/upload/iblock/d5e/av-r-mart.doc
ЗОС, 11.2009 
 http://www.meteorf.ru/upload/iblock/d5f/av-r-noy.doc
ЗОС, 12.2009 
 http://www.meteorf.ru/upload/iblock/e14/av-r-dec.doc
ЗОС, 1.2010 
 http://www.meteorf.ru/upload/iblock/df6/av-r-yanv.doc
ЗОС, 2.2010 
 http://www.meteorf.ru/upload/iblock/812/av-r-fev.doc
ЗОС, 3.2010 
 http://www.meteorf.ru/upload/iblock/329/av-r-mart.doc
ЗОС, 4.2010 
 http://www.meteorf.ru/upload/iblock/1fc/av-r-apr.doc
ЗОС, 6.2010 
 http://www.meteorf.ru/upload/iblock/009/av-r-iun.doc
ЗОС, 7.2010 
 http://www.meteorf.ru/upload/iblock/f94/av-r-iu

IndexError: list index out of range

Удаляем заголовок отчёта без doc(x)-файла

In [60]:
# Remove title, which did not have link on web-site (only html-version, that not parsed.)
# I download this manually
DocTitle.remove('ЗОС, 5.2018')

In [61]:
# Check the len of lists
len(DocTitle), len(DocLinks_wo_Pril)

# First iteration: 134 and 137
# Second iteration: 134 and 133 — I remove links on attachments in cell below
# Third iteration: 133 and 133 — I deleted a document title that did not have a doc(x) file.
# It has an html report, which I downloaded manually.

(133, 133)

### Prepare to download
Готовим всё к скачиванию и скачиваем данные. 

Кодируем часть русскоязычных ссылок из UTF-8 в URL-encoding — иначе `request.urlretrieve` отказывался скачивать файл. На всякий случай делаем новый список с URL-encoding ссылками.

In [62]:
# Prepare links with cyrillic symbols to encode from utf-8 to url-encode
# Otherwice, request.urlretrieve can't download file
DocLinksEnc = DocLinks[:67] + [i[:40] +
                               urllib.parse.quote(i[40:]) for i in DocLinks[67:]]

In [63]:
# Check ex-cyrillic links
DocLinksEnc[67:]

['http://www.meteorf.ru/upload/iblock/f9c/2016-%D0%BD%D0%BE%D1%8F%D0%B1%D1%80%D1%8C-%D0%BC%D0%B5%D1%81%D1%8F%D1%87%D0%BD%D1%8B%D0%B9%20%D0%BE%D0%B1%D0%B7%D0%BE%D1%80%20%D0%BF%D0%BE%20%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8.doc',
 'http://www.meteorf.ru/upload/iblock/e03/2016-%D0%BE%D0%BA%D1%82%D1%8F%D0%B1%D1%80%D1%8C-%D0%BC%D0%B5%D1%81%D1%8F%D1%87%D0%BD%D1%8B%D0%B9%20%D0%BE%D0%B1%D0%B7%D0%BE%D1%80%20%D0%BF%D0%BE%20%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8.doc',
 'http://www.meteorf.ru/upload/iblock/e9c/2016-%D1%81%D0%B5%D0%BD%D1%82%D1%8F%D0%B1%D1%80%D1%8C-%D0%BC%D0%B5%D1%81%D1%8F%D1%87%D0%BD%D1%8B%D0%B9%20%D0%BE%D0%B1%D0%B7%D0%BE%D1%80%20%D0%BF%D0%BE%20%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8.doc',
 'http://www.meteorf.ru/upload/iblock/cf5/2016-%D0%B0%D0%B2%D0%B3%D1%83%D1%81%D1%82-%D0%BC%D0%B5%D1%81%D1%8F%D1%87%D0%BD%D1%8B%D0%B9%20%D0%BE%D0%B1%D0%B7%D0%BE%D1%80%20%D0%BF%D0%BE%20%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8.doc',
 'http://www.meteorf.ru/upload/iblock/cdb/2016-%D0%B8%D1%8E%D0%BB%D1%8C-%D0%BC

Затем создаём из списков словарь и, итерируясь по нему, скачиваем файлы с определёнными названиями в папку `Data`.

При этом делаем проверку по окончанию строки с ссылкой на файл. Если она заканчивается на doc, то скачиваем с расширением `doc`, если нет, то `docx`

In [64]:
# Create dict for download in cycle
DocDict = dict(zip(DocLinksEnc, DocTitle))

In [68]:
# Finally, download with request.urlretrieve
for k, v in DocDict.items():
    DownloadURL = k.replace(' ', '%20')
    DownloadURL = unicodedata.normalize("NFKD", DownloadURL)
    print(k, v)
    if re.search(r'docx', k):
        request.urlretrieve(
            DownloadURL, '/Users/arthu/Documents/Notebooks/INID/Data/' + v + '.docx')
    else:
        request.urlretrieve(
            DownloadURL, '/Users/arthu/Documents/Notebooks/INID/Data/' + v + '.doc')

http://www.meteorf.ru/upload/iblock/72b/mes04r.doc ЗОС, 4.2008
http://www.meteorf.ru/upload/iblock/e89/av-sent.doc ЗОС, 9.2008
http://www.meteorf.ru/upload/iblock/d27/av-noy.doc ЗОС, 11.2008
http://www.meteorf.ru/upload/iblock/a0a/av-dec.doc ЗОС, 12.2008
http://www.meteorf.ru/upload/iblock/52e/av-r-yanv.doc ЗОС, 1.2009
http://www.meteorf.ru/upload/iblock/b3a/av-r-fev.doc ЗОС, 2.2009
http://www.meteorf.ru/upload/iblock/d5e/av-r-mart.doc ЗОС, 3.2009
http://www.meteorf.ru/upload/iblock/d5f/av-r-noy.doc ЗОС, 11.2009
http://www.meteorf.ru/upload/iblock/e14/av-r-dec.doc ЗОС, 12.2009
http://www.meteorf.ru/upload/iblock/df6/av-r-yanv.doc ЗОС, 1.2010
http://www.meteorf.ru/upload/iblock/812/av-r-fev.doc ЗОС, 2.2010
http://www.meteorf.ru/upload/iblock/329/av-r-mart.doc ЗОС, 3.2010
http://www.meteorf.ru/upload/iblock/1fc/av-r-apr.doc ЗОС, 4.2010
http://www.meteorf.ru/upload/iblock/009/av-r-iun.doc ЗОС, 6.2010
http://www.meteorf.ru/upload/iblock/f94/av-r-iul.doc ЗОС, 7.2010
http://www.meteorf.ru/up

http://www.meteorf.ru/upload/iblock/889/2017-%D0%B8%D1%8E%D0%BB%D1%8C-%D0%BC%D0%B5%D1%81%D1%8F%D1%87%D0%BD%D1%8B%D0%B9%20%D0%BE%D0%B1%D0%B7%D0%BE%D1%80%20%D0%BF%D0%BE%20%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8.doc ЗОС, 7.2017
http://www.meteorf.ru/upload/iblock/1ea/%D0%BC%D0%B5%D1%81%D1%8F%D1%87%D0%BD%D1%8B%D0%B9%20%D0%BE%D0%B1%D0%B7%D0%BE%D1%80%20%D0%BF%D0%BE%20%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8_%D0%BC%D0%B0%D0%B9_2017.doc ЗОС, 5.2017
http://www.meteorf.ru/upload/iblock/1b0/2017-%D0%B8%D1%8E%D0%BD%D1%8C-%D0%BC%D0%B5%D1%81%D1%8F%D1%87%D0%BD%D1%8B%D0%B9%20%D0%BE%D0%B1%D0%B7%D0%BE%D1%80%20%D0%BF%D0%BE%20%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8.doc ЗОС, 6.2017
http://www.meteorf.ru/upload/iblock/397/2017-%D0%B0%D0%BF%D1%80%D0%B5%D0%BB%D1%8C-%D0%BC%D0%B5%D1%81%D1%8F%D1%87%D0%BD%D1%8B%D0%B9%20%D0%BE%D0%B1%D0%B7%D0%BE%D1%80%20%D0%BF%D0%BE%20%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8.doc ЗОС, 4.2017
http://www.meteorf.ru/upload/iblock/3f5/2017-%D0%BC%D0%B0%D1%80%D1%82-%D0%BC%D0%B5%D1%81%D1%8F%D1%87%D0%BD

http://www.meteorf.ru/upload/iblock/afe/2018-%D0%B4%D0%B5%D0%BA%D0%B0%D0%B1%D1%80%D1%8C-%D0%BC%D0%B5%D1%81%20%D0%BE%D0%B1%D0%B7%D0%BE%D1%80%20%D0%BF%D0%BE%20%D0%A0%D0%A4.doc ЗОС, 12.2018
http://www.meteorf.ru/upload/iblock/56e/2020-%D0%B4%D0%B5%D0%BA%D0%B0%D0%B1%D1%80%D1%8C-%D0%BC%D0%B5%D1%81%D1%8F%D1%87%D0%BD%D1%8B%D0%B9%20%D0%BE%D0%B1%D0%B7%D0%BE%D1%80%20%D0%BF%D0%BE%20%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B9%D1%81%D0%BA%D0%BE%D0%B9%20%D0%A4%D0%B5%D0%B4%D0%B5%D1%80%D0%B0%D1%86%D0%B8%D0%B8.docx ЗОС, 12.2020
http://www.meteorf.ru/upload/iblock/657/2020-%D0%BD%D0%BE%D1%8F%D0%B1%D1%80%D1%8C-%D0%BC%D0%B5%D1%81%D1%8F%D1%87%D0%BD%D1%8B%D0%B9%20%D0%BE%D0%B1%D0%B7%D0%BE%D1%80%20%D0%BF%D0%BE%20%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B9%D1%81%D0%BA%D0%BE%D0%B9%20%D0%A4%D0%B5%D0%B4%D0%B5%D1%80%D0%B0%D1%86%D0%B8%D0%B8.docx ЗОС, 11.2020
http://www.meteorf.ru/upload/iblock/28a/2020-%D0%BE%D0%BA%D1%82%D1%8F%D0%B1%D1%80%D1%8C-%D0%BC%D0%B5%D1%81%D1%8F%D1%87%D0%BD%D1%8B%D0%B9%20%D0%BE%D0%B1%D0%B7%D0%BE%D1%80%20%

#### Заметка
Если выполнить `python len(DocLinks)`, то получите число 133. Однако в папке Data у нас 133 файла **вместе** с отчётом за май 2018 года, который на сайте Росгидромет есть только в HTML формате **и скрейпером не скачивается.** 

На самом деле ячейка выше скачивает 132 файла, потому что у Росгидромета два раза, под разными URL, выложен один и тот же отчёт —  за декабрь 2012 года. Я проверил, doc-файлы на 100% идентичны. 

Однако, так как я ранее унифицировал названия файлов, то в итоге на компьютер скачивается только один файл. 

Конечно, в случае чего можно написать нечто вроде... 
```python
flipped = {}
  
for k,v in DocDict.items():
    if v not in flipped:
        flipped[v] = [k]
    else:
        flipped[v].append(k)
```
...и тогда мы увидим в перевёрнутом словаре повторяющиеся в основном словаре ключи там, где в перевёртыше больше одного значения; далее, можно проверить доступность ссылок, а затем и идентичность файлов и скачать всё уже по словарю-перевёртышу. 

Однако, мне кажется избыточным вставлять это в код сейчас, когда единственный повтор отловлен. <br> 
Но сама возможность есть. 

Конвертируем `doc` в `docx` с помощью `win32com`

Код взят [отсюда](https://gist.github.com/davecoutts/a6c377d754cf97008f28)

Оставшиеся `doc` удалять не стал, но положил в отдельную папку `Doc` на всячкий случай.

In [74]:
baseDir = 'Data'  # Starting directory for directory walk

word = win32com.client.Dispatch("Word.application")

for dir_path, dirs, files in os.walk(baseDir):
    for file_name in files:

        file_path = os.path.join(dir_path, file_name)
        file_name, file_extension = os.path.splitext(file_path)

        if "~$" not in file_name:
            if file_extension.lower() == '.doc':
                docx_file = '{0}{1}'.format(file_path, 'x')

                # Skip conversion where docx file already exists
                if not os.path.isfile(docx_file):

                    file_path = os.path.abspath(file_path)
                    docx_file = os.path.abspath(docx_file)
                    try:
                        wordDoc = word.Documents.Open(file_path)
                        wordDoc.SaveAs2(docx_file, FileFormat=16)
                        wordDoc.Close()
                    except Exception as e:
                        print('Failed to Convert: {0}'.format(file_path))
                        print(e)